# Bestimmen der zu bestimmenden Größen, die bestimmt bestimmt werden sollen

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from glob import iglob

## Einlesen der Daten

In [2]:
flist = iglob('../Daten/fit/*_params.txt')
df_dict = {}
for f in flist:
    df = pd.read_csv(f, sep='\t', index_col=0)
    df_dict[f[13:-11].replace("_", " ")] = df[~df.index.str.contains('A')]

## Energie aus Geschwindigkeit

In [3]:
def v_to_dE(v):
    """
    Energie aus Geschwindigkeit
    Args:
        v: Geschwindigkeit in mm/s
    Returns:
        Energie in eV
    """
    c = 299792458 # m/s
    E = 14.4e3 # eV
    v = v * 1e-3 # m/s
    return (v/c) * E

## Isomerieverschiebung

In [4]:
iso_dict = {}
print("Isomerieverschiebung")
for key, df in df_dict.items():
    iso_dict[key] = df["Value"][df.index.str.contains('p')].mean()
    print(f"{key:15} {iso_dict[key]:8.5f} mm/s {v_to_dE(iso_dict[key]):15.5e} eV")

Isomerieverschiebung
FePO4            0.28533 mm/s     1.37052e-08 eV
FeSO4            1.15168 mm/s     5.53189e-08 eV
Natural Iron    -0.11298 mm/s    -5.42678e-09 eV
Vacromium       -0.17793 mm/s    -8.54665e-09 eV


## Lebensdauer Vacromium
Jetzt stimmt das hoffentlich

In [5]:
gamma = v_to_dE(df_dict["Vacromium"]["Value"]["g1"]/2) # /2 weil halbwertsbreite in Spektrum doppelt so hoch wie in moessbauer
gamma_err = v_to_dE(df_dict["Vacromium"]["Error"]["g1"]/2) # E ist linear in v
hbar = 6.582119569e-16 # eV*s
tau = hbar/gamma
tau_err = hbar/gamma**2 * gamma_err # Fehlerfortpflanzung a la Gauss
print(f"Lebensdauer Vacromium: {tau:.2e} +- {tau_err:.2e} s")

Lebensdauer Vacromium: 6.07e-08 +- 2.50e-09 s


## Bestimmung des inneren Magnetfelds und des magnetischen Moments μa des angeregten Zustandes von Eisen

Energiekorrekturen durch Hyperfeinstruktur
$$\Delta E_m = -(\vec{\mu} \cdot \vec{B}) = - \frac{m}{I} \mu B$$

### 1. Isomerieshift korrigieren

Im blauen Buch sind $v^*$ die gemessenen Geschwindigkeiten und $v$ die um den Isomerieshift korrigierten Geschwindigkeiten.
$$v_1 > v_2 > v_3 > 0$$
Je zwei Werte für $v_i$, über die gemittelt wird.

In [12]:
#  TODO: Ungenauigkeit fortpflanzen

p_HFS = df_dict["Natural Iron"]["Value"][df_dict["Natural Iron"].index.str.contains('p')] - iso_dict["Natural Iron"]
p_HFS = np.abs(p_HFS)

p_err_HFS = df_dict["Natural Iron"]["Error"][df_dict["Natural Iron"].index.str.contains('p')]

v1, v2, v3 = [(p_HFS[2*i] + p_HFS[2*i+1])/2 for i in range(int(len(p_HFS)/2))]

v1 = v1 * 1e-3 # mm/s -> m/s
v2 = v2 * 1e-3
v3 = v3 * 1e-3

v1_err = v1 * np.sqrt((p_err_HFS[0]/p_HFS[0])**2 + (p_err_HFS[1]/p_HFS[1])**2)
v2_err = v2 * np.sqrt((p_err_HFS[2]/p_HFS[2])**2 + (p_err_HFS[3]/p_HFS[3])**2)
v3_err = v3 * np.sqrt((p_err_HFS[4]/p_HFS[4])**2 + (p_err_HFS[5]/p_HFS[5])**2)

### 2. Welcher Fall aus dem blauen Buch?

In [7]:
THRS = 1e-4 # Schwellwert für Gleichheit
if np.abs(v1 - 2*v2 - v3) < THRS:
    print("1. Fall: A > 0, G < 0, |A| > |G|")
if np.abs(v1 - 2*v2 + v3) < THRS:
    print("2. Fall: A > 0, G < 0, |A| < |G|")
if np.abs(v1 - v2 - 2*v3) < THRS:
    print("3. Fall: A < 0, G < 0, |A| > |G|")
if np.abs(-v1 + v2 + 2*v3) < THRS:
    print("4. Fall: A < 0, G < 0, |G/3| < |A| < |G|")

2. Fall: A > 0, G < 0, |A| < |G|


### 3. $A$,  $G$ und daraus $\mu_\text{a}$, $\mu_\text{g}$ und $B$ bestimmen

$$ \mu_\text{a} \cdot B = \frac{A \cdot I_\text{a} \cdot E_0}{c} \text{ and } \mu_\text{g} \cdot B = \frac{G \cdot I_\text{g} \cdot E_0}{c}$$
$$ \mu_\text{a} = \frac{A \cdot I_\text{a} \cdot \mu_\text{g}}{G \cdot I_\text{g}} \text{ and } B = \frac{G \cdot I_\text{g} \cdot E_0}{c \cdot \mu_\text{g}}$$

In [13]:
# Fall 2:
A = v1 - v2
A_err = np.sqrt(v1_err**2 + v2_err**2)
G = -v2 - v3
G_err = np.sqrt(v2_err**2 + v3_err**2)

Ia = 3/2
Ig = 1/2
c = 299792458 # m/s
E0 = 14.4e3 # eV
mug = 0.0903 # mu_K
mug_err = 0.0007 # mu_K

mua = A * Ia * mug / (G * Ig)
mua_err = mua * np.sqrt((A_err/A)**2 + (G_err/G)**2 + (mug_err/mug)**2)

B = G * Ig * E0 / (c * mug)
B_err = B * np.sqrt((G_err/G)**2 + (mug_err/mug)**2)

print(f"μ_a = {mua:.2e} +- {mua_err:.2e} mu_K")
print(f"B = {B:.2e} +- {B_err:.2e} T")

μ_a = -1.52e-01 +- -2.15e-03 mu_K
B = -1.05e-06 +- -1.19e-08 T


## Bestimmung des elektrischen Feldgradienten des Kristallfelds von Eisenverbindungen

$$v_\text{Q} = v_\text{Q}(\pm 3/2) - v_\text{Q}(\pm 1/2) = \frac{e \cdot Q}{2} \cdot \frac{\partial^2 V}{\partial z^2} \cdot \frac{c}{E_0}$$
$$\frac{\partial^2 V}{\partial z^2} = \frac{2 \cdot E_0 \cdot v_\text{Q}}{e \cdot Q \cdot c}$$

In [9]:
Q = 0.21e-28 # m^2
e = 1.6e-19 # C

names = ["FePO4", "FeSO4"]

pQ = [df_dict[k]["Value"][df_dict[k].index.str.contains('p')] for k in names]

vQ = [pQ[i]["p1"] - pQ[i]["p2"] for i in range(2)]
vQ = np.abs(vQ)

dVdz2 = 2 * E0 * vQ /e /Q /c

for i in range(len(dVdz2)):
    print(names[i], dVdz2[i])

FePO4 2.271287513353445e+43
FeSO4 7.873824847831929e+43


In [10]:
vQ

array([0.79440068, 2.75393219])